# QM7b  
[reference examples](https://github.com/deepchem/deepchem/tree/master/examples/qm7)

In [ ]:
import numpy as np
import tensorflow as tf
import deepchem as dc

In [ ]:
import pathlib
model_dir = pathlib.Path('../models')

**Multitask Network** _(Coulomb Matrix Featurizer)_  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm7/qm7b_tf_model.py)

In [ ]:
qm7b_tasks, qm7b_datasets, transformers = dc.molnet.load_qm7b_from_mat(
    featurizer='CoulombMatrix', split='stratified')
train_dataset, valid_dataset, test_dataset = qm7b_datasets

In [ ]:
from dc.models import MultiTaskRegressor

def mtr_model_builder(model_params, model_dir):
    mtr_model = MultiTaskRegressor(model_dir, **model_params)
    return mtr_model

In [ ]:
fit_transformers [dc.trans.CoulombFitTransformer(train_dataset)]

metrics = [
    dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm7b_tasks),
    "n_features": [23, 23],
    "learning_rate": 0.001,
    "momentum": 0.8,
    "batch_size": 25,
    "weight_init_stddevs": [1 / np.sqrt(400), 1 / np.sqrt(100), 1 / np.sqrt(100)],
    "bias_init_consts": [0.0, 0.0, 0.0],
    "layer_sizes": [400, 100, 100],
    "dropouts": [0.01, 0.01, 0.01],
    "fit_transformers": fit_transformers,
    "n_evals": 10,
}

model = mn_model_builder(params, model_dir)

print("training MTR model")
model.fit(train_dataset, nb_epoch=50)

print("evaluating MTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores: [kcal/mol]")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores: [kcal/mol]")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores: [kcal/mol]")
print(test_scores)

**Kernel Ridge Regression** _(Coulomb Matrix Featurizer)_

In [ ]:
qm7b_tasks, qm7b_datasets, transformers = dc.molnet.load_qm7b_from_mat(
    featurizer='CoulombMatrix', split='stratified')
train_dataset, valid_dataset, test_dataset = qm7b_datasets

In [ ]:
from sklearn.kernel_ridge import KernelRidge

def krr_model_builder(model_params, model_dir):
    krr_model = KernelRidge(model_dir, **model_params)
    sklearn_model_builder = lambda model_dir: dc.models.SklearnModel(krr_model, model_dir)
    return dc.models.SingletaskToMultitask(qm7_tasks, 
                                           sklearn_model_builder, 
                                           model_dir)

In [ ]:
metric = [
    dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")
]

params = {
    "kernel": "rbf",
    "alpha": 5e-4,
    "gamma": 0.008
}

model = krr_model_builder(params, model_dir)

print("training KRR model")
model.fit(train_dataset)

print("evaluating KRR model on {train,valid,test} sets")
train_evaluator = dc.utils.evaluate.Evaluator(model, train_dataset, transformers)
train_scores = train_evaluator.compute_model_performance(metric)
print("train scores: [kcal/mol]")
print(train_scores)

valid_evaluator = dc.utils.evaluate.Evaluator(model, valid_dataset, transformers)
valid_scores = valid_evaluator.compute_model_performance(metric)
print("valid scores: [kcal/mol]")
print(valid_scores)

test_evaluator = dc.utils.evaluate.Evaluator(model, test_dataset, transformers)
test_scores = test_evaluator.compute_model_performance(metric)
print("test scores: [kcal/mol]")
print(test_scores)

**Deep Tensor Neural Network**  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm7/qm7b_DTNN.py)

In [ ]:
qm7b_tasks, qm7b_datasets, transformers = dc.molnet.load_qm7b_from_mat(
    featurizer='Raw', split='stratified')
train_dataset, valid_dataset, test_dataset = qm7b_datasets

In [ ]:
def dtnn_model_builder(model_params, model_dir):
    return dc.models.DTNNModel(output_activation=False,
                               use_queue=False,
                               mode="regression",
                               model_dir=model_dir,
                               **model_params)

In [ ]:
from deepchem.models.tensorgraph.optimizers import ExponentialDecay

metric = [
    dc.metrics.Metric(dc.metric.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm7_tasks),
    "n_embedding": 30,
    "n_hidden": 15,
    "n_distance": 51,
    "distance_min": -1.0,
    "distance_max": 9.2,
    "batch_size": 50,
    "learning_rate": 0.0001
}

model = dtnn_model_builder(params, model_dir)

print("training DTNN model")
model.fit(train_dataset, nb_epoch=1)


print("evaluating DTNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores [kcal/mol]")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores [kcal/mol]")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores [kcal/mol]")
print(test_scores)